In [1]:
from library.ccba import *
import sys
sys.path.insert(0, '/Users/Kwat/binf/code')
from kz_code import *
%matplotlib inline

Using the following packages:
matplotlib v1.5.1
numpy v1.10.4
pandas v0.18.0
rpy2 v2.7.9
scikit-learn v0.17.1
seaborn v0.7.0


In [2]:
# Read gene expressiong
gene_x_sample_df = pd.read_csv('./data/GES24759/GSE24759_series_matrix.txt', sep='\t', comment='!', index_col=0)
print('gene_x_sample_df shape: {}'.format(gene_x_sample_df.shape))

gene_x_sample_df shape: (22944, 211)


In [3]:
# Read gene information
with open('./data/GES24759/GSE24759_family.soft') as f:
    gene_info = {}
    parse_error = {}
    gene_symbol_error = {}
    parse = False
    for i, line in enumerate(f.readlines()):
        line = line.strip()
        if not parse:
            if line == '!platform_table_begin':
                parse = True
                print('Start parsing at line {}'.format(i))
        else:
            if line == '!platform_table_end':
                print('Stop parsing at line {}'.format(i))
                break
            else:
                split = line.split('\t')
                try:
                    if (split[8] == '' or split[8] =='NA'):
                        gene_symbol_error[split[0]] = line
                        continue
                    elif split[0] in gene_info:
                        print('Duplicate at {}'.format(split[0]))
                    else:
                        gene_info[split[0]] = {'gene_title':split[7],
                                               'gene_symbol':split[8].split(' /// '),
                                               'entrez_gene':split[9],
                                               'refseq':split[10].split(' /// ')}
                except:
                    parse_error[split[0]] = line
    print('Parse error: {}'.format(len(parse_error)))
    print('Gene symbol error {}'.format(len(gene_symbol_error)))

Start parsing at line 277
Stop parsing at line 23223
Parse error: 1173
Gene symbol error 741


In [ ]:
# Select k

In [ ]:
# NMF
list_k = list(range(6,7))

nmf_result = {}
for k in list_k:
    print('Perfomring NMF with k {}'.format(k))
    w, h, err =  nmf(gene_x_sample_df, k)
    nmf_result[k] = ({'w':w, 'h':h, 'err':err})

In [ ]:
# Plot NMF result
k=6
ax1 = sns.pointplot(x=list_k, y=[v['err'] for v in nmf_result.values()])
ax1.set(xlabel='k', ylabel='Reconstruction Error')
ax1.set_title('k vs. Reconstruction Error')
plt.show()

In [ ]:
ax2 = sns.heatmap(nmf_result[k]['w'], yticklabels=False)
ax2.set(xlabel='Component', ylabel='Gene')
ax2.set_title('H matrix generated using k={}'.format(k))
plt.show()

In [ ]:
ax3 = sns.heatmap(nmf_result[k]['h'], xticklabels=False)
ax3.set(xlabel='Sample', ylabel='Component')
ax3.set_title('H matrix generated using k={}'.format(k))
plt.show()

In [ ]:
# TODO: figure out why the number of passing values are the same for each col in W
quantile_percent = 0.99
for i in range(nmf_result[k]['w'].shape[1]):
    print('Analyzing component {} ...'.format(i))
    
    col = nmf_result[k]['w'][:, i]
    quantile_threshold = quantile(col, 100/((1-quantile_percent)*100))[-1]
    idx_pass_threshold = np.nonzero(np.greater_equal(col, quantile_threshold)*col)[0]
    print('{} genes above {} quantile threshold.'.format(len(idx_pass_threshold), quantile_percent))

    for gene in gene_x_sample_df.index[idx_pass_threshold]:
        if gene in gene_info:
            pass
        elif gene in parse_error:
            print('Parse error at {}'.format(gene))
        elif gene in gene_symbol_error:
            print('Gene symbol error at {}'.format(gene))
        else:
            print('Unknown error at {}'.format(gene))